In [97]:
from __future__ import division
from __future__ import print_function
import sys

import optproblems.cec2005

if sys.version.startswith('3'):
    xrange = range
import numpy as np
rand = np.random.rand
from scipy.spatial.distance import pdist, squareform

def problem(x):      
    
    global f_otimo
    
    f_otimo = -310
    
    # Função F1 - CEC 2005:
    #cec_f1 = optproblems.cec2005.F1(10)
    #obj = cec_f1(x)
    
    # Função F2 - CEC 2005:
    #cec_f2 = optproblems.cec2005.F2(10)
    #obj = cec_f2(x)
    
    # Função F3 - CEC 2005:
    #cec_f3 = optproblems.cec2005.F3(10)
    #obj = cec_f3(x)
    
    # Função F4 - CEC 2005:
    #cec_f4 = optproblems.cec2005.F4(10)
    #obj = cec_f4(x)
    
    # Função F5 - CEC 2005:
    cec_f5 = optproblems.cec2005.F5(10)
    obj = cec_f5(x)
    
    return obj

class SSA(object):
    def __init__(self, func, 
                 dim = 10,
                 bound = 100,
                 max_iteration = 10000,
                 pop_size = 25,
                 r_a = 1,
                 p_c = 0.7,
                 p_m = 0.1):
        self.func = func
        self.dim = dim
        self.bound = bound
        self.max_iteration = max_iteration
        self.pop_size = pop_size
        self.r_a = r_a
        self.p_c = p_c
        self.p_m = p_m
         
    def run(self, show_info = False):
        self.g_best = np.Inf
        self.g_best_hist = []
        self.g_best_pos = np.zeros(self.dim)
        self.position = rand(self.pop_size, self.dim) * 2 * self.bound - self.bound
        target_position = self.position.copy()
        target_intensity = np.zeros(self.pop_size)
        mask = np.zeros((self.pop_size, self.dim))
        movement = np.zeros((self.pop_size, self.dim))
        inactive = np.zeros(self.pop_size)
        
        if show_info:
            import datetime, time
            print(" " * 15 + "SSA starts at " + datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S'))
            print("=" * 62)
            print(" iter    optimum    pop_min  base_dist  mean_dist time_elapsed")
            print("=" * 62)
            self.start_time = time.time()
        
        iteration = 0
        while (iteration < self.max_iteration):
            iteration += 1
            
            ## Início da alteração
            self.final_position = []
            spider_fitness = []
            bound_inf = -100         # Limite inferior
            bound_sup = 100          # Limite superior
         
            # Loop na posição
            for i in range (0, self.pop_size):
                aux = []
                for j in range(0, self.dim):
                
                    if (self.position[i][j]) < (self.bound * (-1)):
                        aux.append(self.bound*(-1))
                    elif (self.position[i][j]) > (self.bound):
                        aux.append(self.bound)
                    else:
                        aux.append(self.position[i][j])
                
                # Avaliação de uma aranha por vez
                self.final_position = aux
                spider_fitness.append(self.func(self.final_position))
            
            spider_fitness = np.array(spider_fitness)
            # Fim da alteração
            
            base_distance = np.mean(np.std(self.position, 0))
            distance = squareform(pdist(self.position, 'cityblock'))
            
            if np.min(spider_fitness) < self.g_best:
                self.g_best = np.min(spider_fitness)
                self.g_best_pos = self.position[np.argmin(spider_fitness)].copy()
            self.g_best_hist.append(self.g_best)
            if show_info and (iteration == 1 or iteration == 10
                    or (iteration < 1001 and iteration % 100 == 0) 
                    or (iteration < 10001 and iteration % 1000 == 0)
                    or (iteration < 100000 and iteration % 10000 == 0)):
                elapsed_time = time.time() - self.start_time
                print(repr(iteration).rjust(5), "%.4e" % self.g_best, "%.4e" % np.min(spider_fitness),
                      "%.4e" % base_distance, "%.4e" % np.mean(distance), 
                      "%02d:%02d:%02d.%03d" % (elapsed_time // 3600, elapsed_time // 60 % 60, 
                                               elapsed_time % 60, (elapsed_time % 1) * 1000))
            
            # Alteração na Função aptidão -> somar valor ótimo para não resultar em valores menores que 0.
            intensity_source = np.log(1. / ((spider_fitness + (f_otimo * (-1))) + 1E-100) + 1)            
            intensity_attenuation = np.exp(-distance / (base_distance * self.r_a))
            intensity_receive = np.tile(intensity_source, self.pop_size).reshape(self.pop_size, self.pop_size) * intensity_attenuation
            
            max_index = np.argmax(intensity_receive, axis = 1)
            keep_target = intensity_receive[np.arange(self.pop_size),max_index] <= target_intensity
            keep_target_matrix = np.repeat(keep_target, self.dim).reshape(self.pop_size, self.dim)
            inactive = inactive * keep_target + keep_target
            target_intensity = target_intensity * keep_target + intensity_receive[np.arange(self.pop_size),max_index] * (1 - keep_target)
            target_position = target_position * keep_target_matrix + self.position[max_index] * (1 - keep_target_matrix)
            
            rand_position = self.position[np.floor(rand(self.pop_size * self.dim) * self.pop_size).astype(int), \
                np.tile(np.arange(self.dim), self.pop_size)].reshape(self.pop_size, self.dim)
            new_mask = np.ceil(rand(self.pop_size, self.dim) + rand() * self.p_m - 1)
            keep_mask = rand(self.pop_size) < self.p_c**inactive
            inactive = inactive * keep_mask
            keep_mask_matrix = np.repeat(keep_mask, self.dim).reshape(self.pop_size, self.dim)
            mask = keep_mask_matrix * mask + (1 - keep_mask_matrix) * new_mask
                            
            follow_position = mask * rand_position + (1 - mask) * target_position
            movement = np.repeat(rand(self.pop_size), self.dim).reshape(self.pop_size, self.dim) * movement + \
                (follow_position - self.position) * rand(self.pop_size, self.dim)
            self.position = self.position + movement
            
        if show_info:
            elapsed_time = time.time() - self.start_time
            print("=" * 62)
            print(repr(iteration).rjust(5), "%.4e" % self.g_best, "%.4e" % np.min(spider_fitness),
                  "%.4e" % base_distance, "%.4e" % np.mean(distance), 
                  "%02d:%02d:%02d.%03d" % (elapsed_time // 3600, elapsed_time // 60 % 60, 
                                           elapsed_time % 60, (elapsed_time % 1) * 1000))
            print("=" * 62)
        return {'global_best_fitness': self.g_best,
                'global_best_solution': self.g_best_pos,
                'iterations': iteration + 1}

if __name__ == '__main__':
    SSA(problem).run(True)

               SSA starts at 2020-11-24 20:20:37
 iter    optimum    pop_min  base_dist  mean_dist time_elapsed
    1 2.4120e+04 2.4120e+04 5.8599e+01 6.6759e+02 00:00:00.005
   10 2.0704e+04 2.1424e+04 5.7629e+01 6.5552e+02 00:00:00.040
  100 1.1598e+04 1.1598e+04 5.8019e+01 6.5202e+02 00:00:00.350
  200 3.3983e+03 3.3983e+03 6.0442e+01 6.6900e+02 00:00:00.669
  300 2.7097e+03 2.7097e+03 6.5997e+01 7.0072e+02 00:00:01.057
  400 2.4535e+03 2.4535e+03 6.3849e+01 6.5788e+02 00:00:01.457
  500 2.2604e+03 2.2604e+03 6.0275e+01 6.1854e+02 00:00:01.869
  600 9.5400e+02 1.1204e+03 5.2050e+01 5.5389e+02 00:00:02.249
  700 3.6396e+02 3.7032e+02 4.5357e+01 4.7921e+02 00:00:02.609
  800 -4.7528e+01 -4.7528e+01 3.8897e+01 4.0806e+02 00:00:02.979
  900 -9.1966e+01 -9.1966e+01 4.1580e+01 4.3189e+02 00:00:03.319
 1000 -1.1554e+02 -1.0519e+02 4.0316e+01 4.0625e+02 00:00:03.719
 2000 -3.0999e+02 -3.0999e+02 1.5295e+01 1.5839e+02 00:00:07.919
 3000 -3.1000e+02 -3.1000e+02 1.5758e+01 1.7041e+02 00:00:13.